In [1]:
from itertools import count, cycle, islice
from collections import namedtuple, defaultdict, deque, Counter

def Input(day):
    "Open this day's input file."
    filename = 'input{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        print("No input file found for day", day)
        
def consume(iterator, n):
    "Advance the iterator n-steps ahead. If n is none, consume entirely."
    # Use functions that consume iterators at C speed.
    if n is None:
        # feed the entire iterator into a zero-length deque
        collections.deque(iterator, maxlen=0)
    else:
        # advance to the empty slice starting at position n
        next(islice(iterator, n, n), None)

# [Day 1](http://adventofcode.com/2017/day/1): Inverse Captcha
### Part one
Find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

In [2]:
def parse(text):
    "Parse a string of numbers into a list of digits"
    return [int(n) for n in list(text.strip())]

def captcha_n(digits, k):
    sum = 0
    
    for i, n in enumerate(digits):
        if n == digits[(i + k) % len(digits)]:
            sum += n
    
    return sum

def captcha(digits):
    return captcha_n(digits, 1)

assert parse("1234") == [1, 2, 3, 4]
assert captcha([1, 1, 2, 2]) == 3
assert captcha([1, 1, 1, 1]) == 4
assert captcha([1, 2, 3, 4]) == 0
assert captcha([9, 1, 2, 1, 2, 1, 2, 9]) == 9

captcha(parse(Input(1).read()))

1034

#### Part two
Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list.

In [3]:
def captcha_halfway(digits):
    assert len(digits) % 2 == 0
    return captcha_n(digits, len(digits) // 2)

assert captcha_halfway([1, 2, 1, 2]) == 6
assert captcha_halfway([1, 2, 3, 4, 2, 5]) == 4
assert captcha_halfway([1, 2, 3, 1, 2, 3]) == 12
assert captcha_halfway([1, 2, 1, 3, 1, 4, 1, 5]) == 4

captcha_halfway(parse(Input(1).read()))

1356

# [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum
### Part one
For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

In [4]:
def max_range(row):
    low = float("inf")
    high = 0
    
    for i in row:
        if i < low:
            low = i
        if i > high:
            high = i
    
    return high - low

def checksum(file, row_func):
    sum = 0
    for row in file:
        sum += row_func(row)
        
    return sum

def parse(text):
    "Parse lines of tab separated numbers into lists of integers"
    res = []
    for line in text.strip().split("\n"):
        res.append([int(n) for n in line.split("\t")])
    return res

assert parse("1236\t741\t557\t1029\t144") == [[1236, 741, 557, 1029, 144]]
test_input = [[5, 1, 9, 5], [7, 5, 3], [2, 4, 6, 8]]
assert checksum(test_input, max_range) == 18

checksum(parse(Input(2).read()), max_range)

42299

### Part two
It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

In [5]:
def quotient(row):
    row.sort()
    for i, n in enumerate(row):
        for m in row[:i:-1]:
            if m % n == 0:
                return m // n

assert checksum([[5, 9, 2, 8]], quotient) == 4

checksum(parse(Input(2).read()), quotient)

277

# [Day 3](http://adventofcode.com/2017/day/3): Spiral Memory
### Part one

You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```
While this is very space-efficient (no squares are skipped), requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square 1.

For example:

Data from square 1 is carried 0 steps, since it's at the access port.
Data from square 12 is carried 3 steps, such as: down, left, left.
Data from square 23 is carried only 2 steps: up twice.
Data from square 1024 must be carried 31 steps.
How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?

---

So, I could either build out the entire spiral and walk it to find the result, _or_ I could just do some math. We could find the corner nearest to our destination, and easily compute how many steps away that is from the center. It's then easy to see how many steps the corner is from our destination. In other words, 
``` 
distance(center, destination) == distance(center, nearest_corner) - distance(nearest_corner, destination)
```
Math sounds faster. 

In [6]:
def center_distance(n):
    "Given a number, return its distance from the center of the spiral map"
    box_height = 1
    x = 0
    
    # Find box height
    while x < n:
        box_height += 2
        x = box_height * box_height
    
    # Find nearest corner
    corner = x
    while corner > n:
        corner -= box_height - 1
    
    if n - corner > corner + (box_height - 1) - n:
        corner = corner + (box_height - 1)
    
    # Distance from center to nearest corner - distance from nearest corner to point
    diag = (box_height // 2) * 2
    print(diag - abs(corner - n))

center_distance(265149)


438


### Part 2
As a stress test on the system, the programs here clear the grid and then store the value 1 in square 1. Then, in the same allocation order as shown above, they store the sum of the values in all adjacent squares, including diagonals.

So, the first few squares' values are chosen as follows:

Square 1 starts with the value 1.
Square 2 has only one adjacent filled square (with value 1), so it also stores 1.
Square 3 has both of the above squares as neighbors and stores the sum of their values, 2.
Square 4 has all three of the aforementioned squares as neighbors and stores the sum of their values, 4.
Square 5 only has the first and fourth squares as neighbors, so it gets the value 5.
Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:

```
147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...
```
What is the first value written that is larger than your puzzle input?

---

Whoops, should have built the spiral.

In [7]:
class SpiralLocation:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.value = 1

def get_neighbors(x, y):
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))
        
def spiral(end, cell_func):
    "Build a spiraled map from 1..end, setting the value at each cell with cell_func"
    loc = SpiralLocation()
    width = 1   
    spiral = {(loc.x, loc.y): loc.value}

    down = lambda x: x - 1
    up = lambda x: x + 1
    
    def spiral_side(axis, direction):
        corner = max_coord if direction == up else -max_coord
        current = getattr(loc, axis)

        while current != corner:
            setattr(loc, axis, direction(current))
            loc.value = cell_func(loc, spiral)
            spiral[(loc.x, loc.y)] = loc.value
            if loc.value >= end:
                return True
            current = getattr(loc, axis)
        return False
        
    while loc.value < end:
        width += 2
        max_coord = width // 2
        
        if spiral_side("x", up):
            return spiral
        if spiral_side("y", up):
            return spiral
        if spiral_side("x", down):
            return spiral
        if spiral_side("y", down):
            return spiral
        if spiral_side("x", up):
            return spiral
            
    return spiral

def neighbor_sum(loc, spiral):
    res = 0
    neighbor_coords = get_neighbors(loc.x, loc.y)
    for n in neighbor_coords:
        value = spiral.get(n)
        if value:
            res += value

    return res
 
res = spiral(25, lambda loc, spiral: loc.value + 1)
assert res[(0, 2)] == 15
assert res[(0, 0)] == 1
assert res[(2, -2)] == 25
assert res[(-1, 0)] == 6

test_spiral = {(0, 1): 4, (-1, 1): 5, (0, 0): 1, (-1, 0): 6, (-1, -1): 7, (0, -1): 8, (1, 0): 2, (1, -1): 9, (1, 1): 3}
test_loc = SpiralLocation()
test_loc.x = -1
test_loc.y = 0
assert neighbor_sum(test_loc, test_spiral) == 25

neighbor_spiral = spiral(750, neighbor_sum)
assert sorted(list(neighbor_spiral.values()))[-1] == 806

neighbor_spiral = spiral(265149, neighbor_sum)
sorted(list(neighbor_spiral.values()))[-1]

266330

# [Day 4](http://adventofcode.com/2017/day/4): High-Entropy Passphrases

### Part one
A new system policy has been put in place that requires all accounts to use a passphrase instead of simply a password. A passphrase consists of a series of words (lowercase letters) separated by spaces.

To ensure security, a valid passphrase must contain no duplicate words.

For example:
```
aa bb cc dd ee is valid.
aa bb cc dd aa is not valid - the word aa appears more than once.
aa bb cc dd aaa is valid - aa and aaa count as different words.
```
The system's full passphrase list is available as your puzzle input. How many passphrases are valid?

In [8]:
def no_dupes(words):
    return len(words) - len(set(words)) == 0

def parse_words(text):
    return text.split(" ")

def valid_passphrase_count(lines, filter_func):
    parsed_lines = map(parse_words, lines)
    return len(list(filter(filter_func, parsed_lines)))


assert no_dupes(["aa", "bb", "cc", "dd", "ee"]) == True
assert no_dupes(["aa", "bb", "cc", "dd", "aa"]) == False
assert no_dupes(["aa", "bb", "cc", "dd", "aaa"]) == True
assert parse_words("aa bb cc") == ["aa", "bb", "cc"]

valid_passphrase_count(Input(4).read().splitlines(), no_dupes)

383

### Part two
For added security, yet another system policy has been put in place. Now, a valid passphrase must contain no two words that are anagrams of each other - that is, a passphrase is invalid if any word's letters can be rearranged to form any other word in the passphrase.

For example:
```
abcde fghij is a valid passphrase.
abcde xyz ecdab is not valid - the letters from the third word can be rearranged to form the first word.
a ab abc abd abf abj is a valid passphrase, because all letters need to be used when forming another word.
iiii oiii ooii oooi oooo is valid.
oiii ioii iioi iiio is not valid - any of these words can be rearranged to form any other word.
```
Under this new system policy, how many passphrases are valid?

In [9]:
def no_anagrams(words):
    sorted_words = [''.join(s) for s in list(map(sorted, words))]
    return no_dupes(sorted_words)

assert no_anagrams(["cat", "act"]) == False
valid_passphrase_count(Input(4).read().splitlines(), no_anagrams)

265

# [Day 5](http://adventofcode.com/2017/day/5): A Maze of Twisty Trampolines, All Alike

### Part one
An urgent interrupt arrives from the CPU: it's trapped in a maze of jump instructions, and it would like assistance from any programs with spare cycles to help find the exit.

The message includes a list of the offsets for each jump. Jumps are relative: -1 moves to the previous instruction, and 2 skips the next one. Start at the first instruction in the list. The goal is to follow the jumps until one leads outside the list.

In addition, these instructions are a little strange; after each jump, the offset of that instruction increases by 1. So, if you come across an offset of 3, you would move three instructions forward, but change it to a 4 for the next time it is encountered.

For example, consider the following list of jump offsets:

```
0
3
0
1
-3
```
Positive jumps ("forward") move downward; negative jumps move upward. For legibility in this example, these offset values will be written all on one line, with the current instruction marked in parentheses. The following steps would be taken before an exit is found:

```
(0) 3  0  1  -3  - before we have taken any steps.
(1) 3  0  1  -3  - jump with offset 0 (that is, don't jump at all). Fortunately, the instruction is then incremented to 1.
 2 (3) 0  1  -3  - step forward because of the instruction we just modified. The first instruction is incremented again, now to 2.
 2  4  0  1 (-3) - jump all the way to the end; leave a 4 behind.
 2 (4) 0  1  -2  - go back to where we just were; increment -3 to -2.
 2  5  0  1  -2  - jump 4 steps forward, escaping the maze.
 ```
In this example, the exit is reached in 5 steps.

How many steps does it take to reach the exit?

In [10]:
part_one = lambda x: x + 1

def trampoline(jumps, rule):
    loc = 0
    jump_count = 0
    while True:
        # index of next hop
        move = loc + jumps[loc]
        if move < 0 or move >= len(jumps):
            return jump_count + 1
        jumps[loc] = rule(jumps[loc])
        jump_count += 1
        loc = move

assert trampoline([0, 3, 0, 1, -3], part_one) == 5
trampoline([int(n) for n in Input(5).readlines()], part_one)

351282

Now, the jumps are even stranger: after each jump, if the offset was three or more, instead decrease it by 1. Otherwise, increase it by 1 as before.

Using this rule with the above example, the process now takes 10 steps, and the offset values after finding the exit are left as 2 3 2 3 -1.

How many steps does it now take to reach the exit?

In [11]:
part_two = lambda x: x - 1 if x > 2 else x + 1
assert trampoline([0, 3, 0, 1, -3], part_two) == 10
trampoline([int(n) for n in Input(5).readlines()], part_two)

24568703

# [Day 6](http://adventofcode.com/2017/day/6): Memory Reallocation

### Part one

A debugger program here is having an issue: it is trying to repair a memory reallocation routine, but it keeps getting stuck in an infinite loop.

In this area, there are sixteen memory banks; each memory bank can hold any number of blocks. The goal of the reallocation routine is to balance the blocks between the memory banks.

The reallocation routine operates in cycles. In each cycle, it finds the memory bank with the most blocks (ties won by the lowest-numbered memory bank) and redistributes those blocks among the banks. To do this, it removes all of the blocks from the selected bank, then moves to the next (by index) memory bank and inserts one of the blocks. It continues doing this until it runs out of blocks; if it reaches the last memory bank, it wraps around to the first one.

The debugger would like to know how many redistributions can be done before a blocks-in-banks configuration is produced that has been seen before.

For example, imagine a scenario with only four memory banks:

- The banks start with 0, 2, 7, and 0 blocks. 
- The third bank has the most blocks, so it is chosen for redistribution.
- Starting with the next bank (the fourth bank) and then continuing to the first bank, the second bank, and so on, the 7 blocks are spread out over the memory banks. The fourth, first, and second banks get two blocks each, and the third bank gets one back. The final result looks like this: 2 4 1 2.
- Next, the second bank is chosen because it contains the most blocks (four). Because there are four memory banks, each gets one block. The result is: 3 1 2 3.
- Now, there is a tie between the first and fourth memory banks, both of which have three blocks. The first bank wins the tie, and its three blocks are distributed evenly over the other three banks, leaving it with none: 0 2 3 4.
- The fourth bank is chosen, and its four blocks are distributed such that each of the four banks receives one: 1 3 4 1.
 -The third bank is chosen, and the same thing happens: 2 4 1 2.
 -At this point, we've reached a state we've seen before: 2 4 1 2 was already seen. The infinite loop is detected after the fifth block redistribution cycle, and so the answer in this example is 5.

Given the initial block counts in your puzzle input, how many redistribution cycles must be completed before a configuration is produced that has been seen before?

In [12]:
INPUT = "0    5    10    0    11    14    13    4    11    8    8    7    1    4    12    11"

def redistribute(banks):
    block_count = max(banks)
    loc = banks.index(block_count)
    banks[loc] = 0
    
    while block_count > 0:
        loc = (loc + 1) % len(banks)
        banks[loc] += 1
        block_count -= 1

b = [0, 2, 7, 0]
redistribute(b)
assert b == [2, 4, 1, 2]

def find_loop(banks):
    seen_configs = set()
    cycles = 0
    
    while True:
        config = ".".join([str(n) for n in banks])
        if config in seen_configs:
            return cycles
        seen_configs.add(config)
        redistribute(banks)
        cycles += 1
    
assert find_loop([0, 2, 7, 0]) == 5

find_loop([int(n) for n in INPUT.split("    ")])

7864

### Part two

Out of curiosity, the debugger would also like to know the size of the loop: starting from a state that has already been seen, how many block redistribution cycles must be performed before that same state is seen again?

In the example above, 2 4 1 2 is seen again after four cycles, and so the answer in that example would be 4.

How many cycles are in the infinite loop that arises from the configuration in your puzzle input?

In [13]:
def find_loop_size(banks):
    seen_configs = {}
    cycles = 0
    
    while True:
        config = ".".join([str(n) for n in banks])
        if config in seen_configs:
            return cycles - seen_configs[config]
        seen_configs[config] = cycles
        redistribute(banks)
        cycles += 1

assert find_loop_size([0, 2, 7, 0]) == 4
find_loop_size([int(n) for n in INPUT.split("    ")])

1695

# [Day 7](http://adventofcode.com/2017/day/7): Recursive Circus

### Part one

In [14]:
test_input = """pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)"""

class Disc:
    def __init__(self, name, weight, children):
        self.name = name
        self.weight = weight
        self.children = children
        self.original_weight = weight

def parse_disc(text):
    parts = text.strip().split(" ")
    name, weight, *_ = parts
    weight = int(weight[1:-1])
    children = [child.strip(',') for child in parts[3:]]
    return Disc(name, weight, children)

test_disc = parse_disc(test_input.split("\n")[7])
assert test_disc.name == 'padx'
assert test_disc.weight == 45
assert test_disc.children == ['pbga', 'havc', 'qoyq']

def find_root(discs):
    disc_names = {disc.name for disc in discs}
    children = {child for disc in discs for child in disc.children}
    return disc_names.difference(children).pop()
    
test_discs = [parse_disc(t) for t in test_input.split("\n")]
assert find_root(test_discs) == 'tknk'

discs = [parse_disc(t) for t in Input(7).readlines()]
find_root(discs)

'vmpywg'

### Part two

In [15]:
def build_tree(root, discs):
    root.children = [discs[name] for name in root.children]
    for child in root.children:
        build_tree(child, discs)

def parse_discs(text):
    discs = [parse_disc(t) for t in text.split("\n")]
    root = find_root(discs)
    disc_map = {disc.name: disc for disc in discs}
    build_tree(disc_map[root], disc_map)
    return disc_map[root]

def build_weights(root, path=[]):
    path.append(root)
    for child in root.children:
        for node in path:
            node.weight += child.weight
        build_weights(child)
    path.pop()
        
root = parse_discs(test_input)
build_weights(root)
assert root.weight == 778
assert root.children[0].weight == 251
assert root.children[1].weight == 243
assert root.children[0].children[0].weight == 61    
    
def correct_wrong(weighted_root):
    weights = {}
    for child in weighted_root.children:
        if weights.get(child.weight):
            weights[child.weight].append(child)
        else:
            weights[child.weight] = [child]

    outlier = None
    for groups in weights.values():
        if len(groups) == 1:
            outlier = groups[0]
        else:
            matching = groups[0]
            
    if len(set(c.weight for c in outlier.children)) == 1:
        return outlier.original_weight - (outlier.weight - matching.weight)
    else:
        return correct_wrong(outlier)

assert correct_wrong(root) == 60

root = parse_discs(Input(7).read().strip())
build_weights(root)
print(correct_wrong(root))

1674


# [Day 8](http://adventofcode.com/2017/day/8): I Heard You Like Registers

Using Python for this feels like cheating...

In [16]:
test_input = """b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10"""

operation = {
    "inc": lambda x, y: x + y,
    "dec": lambda x, y: x - y
}

condition = {
    ">": lambda x, y: x > y,
    "<": lambda x, y: x < y,
    ">=": lambda x, y: x >= y,
    "<=": lambda x, y: x <= y,
    "==": lambda x, y: x == y,
    "!=": lambda x, y: x != y
}

def parse_reg(line, regs):
    target, symbol, amount, _, cond_target, cond_symbol, cond_amount = line.split()
    ret = 0
    if condition[cond_symbol](regs[cond_target], int(cond_amount)):
        regs[target] = operation[symbol](regs[target], int(amount))
        ret = regs[target]
    return ret

def max_reg(text):
    historical_max = 0
    regs = defaultdict(int)
    for line in text.splitlines():
        historical_max = max(historical_max, parse_reg(line, regs))
    return max(regs.values()), historical_max

assert max_reg(test_input) == (1, 10)

max_reg(Input(8).read())

(4647, 5590)

# [Day 9](http://adventofcode.com/2017/day/9): Stream Processing

Today was my first time being bitten by mutable default args 😱

I thought building a tree of the input might be useful for whatever Part 2 would entail, but that would up being straightforward.

In [17]:
class Node:
    def __init__(self, depth):
        self.depth = depth
        self.children = []
        
def parse_garbage(it):
    count = 0
    for c in it:
        if c == '>':
            return count
        if c == '!':
            next(it)
            continue
        count += 1

def parse_stream(it, root=None):
    if root == None:
        root = Node(0)
    for c in it:
        if c == '<':
            parse_garbage(it)
        elif c == '{':
            root.children.append(parse_stream(it, Node(root.depth + 1)))
        elif c == '}':
            return root
            
    return root
        
def walk_tree(root):
    score = 0
    nodes = deque([root])
    while nodes:
        node = nodes.popleft()
        score += node.depth
        for child in node.children:
            nodes.append(child)
    return score

def find_score(text):
    return walk_tree(parse_stream(iter(text)))

assert find_score("{}") == 1
assert find_score("{{{}}}") == 6
assert find_score("{{},{}}") == 5
assert find_score("{{{},{},{{}}}}") == 16.
assert find_score("{<a>,<a>,<a>,<a>}") == 1
assert find_score("{{<ab>},{<ab>},{<ab>},{<ab>}}") == 9
assert find_score("{{<!!>},{<!!>},{<!!>},{<!!>}}") == 9
assert find_score("{{<a!>},{<a!>},{<a!>},{<ab>}}") == 3

find_score(Input(9).read())

12803

In [18]:
def count_garbage(it):
    count = 0
    for c in it:
        if c == '<':
            count += parse_garbage(it)
    return count

assert count_garbage(iter('<{o"i!a,<{i<a>')) == 10

count_garbage(iter(Input(9).read()))

6425

# [Day 10](http://adventofcode.com/2017/day/10): Knot hash

In hindsight, itertools.cycle would have been real useful here.

In [19]:
INPUT = [120,93,0,90,5,80,129,74,1,165,204,255,254,2,50,113]

def ring_get(ring, n, i):
    "Get n elements from the ring starting at index i"
    to_end = ring[i:min(n + i, len(ring))]
    wrap = ring[:n - len(to_end)]
    return to_end + wrap
    
def ring_put(ring, elements, i):
    "Insert elements in ring, starting at index i"
    for j, e in enumerate(elements):
        idx = (i + j) % len(ring)
        ring[idx] = e
    return ring
    
def list_gen(n):
    "Generate a list of numbers from 0..n"
    return [i for i in range(n + 1)]

def twist(ring, length, i):
    to_reverse = ring_get(ring, length, i)
    to_reverse.reverse()
    ring_put(ring, to_reverse, i)
    return ring
    
def do_twists(ring, lengths, pos=0, skip_size=0):
    for l in lengths:
        twist(ring, l, pos)
        pos = (pos + l + skip_size) % len(ring)
        skip_size += 1
    return (ring, pos, skip_size)
    
l = [1, 2, 3, 4]
assert ring_get(l, 4, 2) == [3, 4, 1, 2]
assert ring_put(l, [6, 7, 8], 2) == [8, 2, 6, 7]
assert list_gen(4) == [0, 1, 2, 3, 4]

test_input = [3, 4, 1, 5]
test_list = list_gen(4)
assert twist(test_list, 3, 0) == [2, 1, 0, 3, 4]

assert do_twists(list_gen(4), test_input)[0] == [3, 4, 2, 1, 0]

first, second, *_ = do_twists(list_gen(255), INPUT)[0]
first * second

826

In [20]:
SUFFIX = [17, 31, 73, 47, 23]
ASCII_INPUT = "120,93,0,90,5,80,129,74,1,165,204,255,254,2,50,113"
RING_SIZE = 255
ITERATIONS = 64
BLOCK = 16

def hex_format(c):
    res = format(c, 'x')
    if len(res) == 1:
        return "0" + res
    return res

def knot_hash(s):
    lengths = [ord(c) for c in s] + SUFFIX
    pos = 0
    skip_size = 0
    sparse_hash = list_gen(RING_SIZE)
    for i in range(0, ITERATIONS):
        sparse_hash, pos, skip_size = do_twists(sparse_hash, lengths, pos, skip_size)
    
    dense_hash = []
    chunks = [sparse_hash[i:i + BLOCK] for i in range(0, len(sparse_hash), BLOCK)]
    for chunk in chunks:
        res = 0
        for el in chunk:
            res = res ^ el
        dense_hash.append(hex_format(res))
    
    return "".join(dense_hash)

assert knot_hash("") == "a2582a3a0e66e6e86e3812dcb672a272"
assert knot_hash("AoC 2017") == "33efeb34ea91902bb2f59c9920caa6cd"

knot_hash(ASCII_INPUT)

'd067d3f14d07e09c2e7308c3926605c4'

# [Day 11](http://adventofcode.com/2017/day/11): Hex Ed

Approaching part 1 by enumerating all direction combinations that cancel out or reduce to a simpler direction.

In [21]:
def shortest_path(hex_path):    
    def equivalent(d1, d2, d3):
        for i in range(min(counts[d1], counts[d2])):
            counts[d1] -= 1
            counts[d2] -= 1
            
            if d3:
                counts[d3] += 1
            
    steps = hex_path.split(",")
    counts = Counter(steps)
             
    equivalent("ne", "sw", None)
    equivalent("se", "nw", None)
    equivalent("s", "n", None)

    equivalent("ne", "nw", "n")
    equivalent("se", "sw", "s")
    equivalent("ne", "s", "se")
    equivalent("nw", "s", "sw")
    equivalent("sw", "n", "nw")
    equivalent("se", "n", "ne")
    return sum(counts.values())
        
assert shortest_path("ne,ne,ne") == 3
assert shortest_path("ne,ne,sw,sw") == 0
assert shortest_path("ne,ne,s,s") == 2
assert shortest_path("se,sw,se,sw,sw") == 3

shortest_path(Input(11).read().strip())

722

Whoops, can't reuse any of this for part 2.

In [22]:
class Location:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def move(self, direction):
        if direction == "n":
            self.y += 2
        if direction == "s":
            self.y -= 2
        if direction == "ne":
            self.x += 1
            self.y += 1
        if direction == "se":
            self.x += 1
            self.y -= 1
        if direction == "nw":
            self.x -= 1
            self.y += 1
        if direction == "sw":
            self.x -= 1
            self.y -= 1

def furthest_from_start(hex_path):
    loc = Location(0, 0)
    steps = hex_path.split(",")
    furthest = 0
    for step in steps:
        loc.move(step)
        furthest = max(furthest, (abs(loc.x) + abs(loc.y)) // 2)
    return furthest

assert furthest_from_start("ne,ne,sw,sw") == 2
furthest_from_start(Input(11).read().strip())

1551

# [Day 12](http://adventofcode.com/2017/day/12): Digital Plumber


In [23]:
test_input = """0 <-> 2
1 <-> 1
2 <-> 0, 3, 4
3 <-> 2, 4
4 <-> 2, 3, 6
5 <-> 6
6 <-> 4, 5"""
 
def piped(text, source):
    def add_deps(key):
        for e in pipes[key]:
            if not e in connected:
                connected.add(e)
                add_deps(e)
    pipes = {}
    connected = set()
    for line in text.split("\n"):
        key, _, *values = line.split(" ")
        values = set([v.strip(',') for v in values])
        pipes[key] = values
              
    add_deps(source)

    return len(connected)
 
assert piped(test_input, "0") == 6

piped(Input(12).read().strip(), "0")


145

In [24]:
def groups(text):
    def add_deps(key, connected=None):
        if not connected:
            connected = set()
        for e in pipes[key]:
            if not e in connected:
                connected.add(e)
                add_deps(e, connected)
        return connected
                
    pipes = {}
    groups = set()
    for line in text.split("\n"):
        key, _, *values = line.split(" ")
        values = set([v.strip(',') for v in values])
        pipes[key] = values
              
    groups = set()
    for k in pipes:
        deps = add_deps(k)
        groups.add(frozenset(deps))

    return len(groups)


assert groups(test_input) == 2
groups(Input(12).read().strip())


207

# [Day 13](http://adventofcode.com/2017/day/13): Packet Scanners

In [ ]:
INPUT = """0: 4
1: 2
2: 3
4: 4
6: 8
8: 5
10: 6
12: 6
14: 10
16: 8
18: 6
20: 9
22: 8
24: 6
26: 8
28: 8
30: 12
32: 12
34: 12
36: 12
38: 10
40: 12
42: 12
44: 14
46: 8
48: 14
50: 12
52: 14
54: 14
58: 14
60: 12
62: 14
64: 14
66: 12
68: 12
72: 14
74: 18
76: 17
86: 14
88: 20
92: 14
94: 14
96: 18
98: 18"""

TEST_INPUT = """0: 3
1: 2
4: 4
6: 4"""

class Layer:
    def __init__(self, depth, range):
        self.depth = depth
        self.range = range
        self.scanner_location = 0
        self.up = lambda x: x + 1
        self.down = lambda x: x - 1
        self.scanner_move = self.up

    def bump_scanner(self):
        self.scanner_location = self.scanner_move(self.scanner_location)
        if self.scanner_location == 0:
            self.scanner_move = self.up
        if self.scanner_location == self.range - 1:
            self.scanner_move = self.down

class Firewall:
    def __init__(self, text):
        self.layers = {}
        self.packet = -1
        for line in text.splitlines():
            depth, range = [int(part.strip()) for part in line.split(":")]
            self.layers[depth] = Layer(depth, range)
            self.last = depth

    def reset_scanners(self):
        for layer in self.layers.values():
            layer.scanner_location = 0
            layer.scanner_move = layer.up
            layer.direction = "UP"
            
    def move_scanners(self):
        for layer in self.layers.values():
            layer.bump_scanner()

    def move_packet(self):
        self.packet += 1
        layer = self.layers.get(self.packet)
        loc = None
        if layer:
            loc = layer.scanner_location
        if layer and layer.scanner_location == 0:
            return layer.depth * layer.range
        return 0

    def cross(self):
        severity = 0
        for i in range(self.last + 1):
            severity += self.move_packet()
            self.move_scanners()
        return severity
            
f = Firewall(TEST_INPUT)
assert f.cross() == 24

f = Firewall(INPUT)
print("result", f.cross())


result 1844


Building the simulation above took way too long to compute part 2, so instead I'm now just calculating the position of the scanner at each timer, layer combo.

In [ ]:
def find_delay(text):
    layers = {}
    for line in text.splitlines():
        depth, length = [int(part.strip()) for part in line.split(":")]
        layers[depth] = length
        
    for delay in count(0):
        if find_hit(delay, layers):
            return delay

def find_hit(delay, layers):
    highest = max(layers.keys())
    time = delay
    for tick in range(highest + 1):
        time = delay + tick
        layer = layers.get(tick)
        if layer and scanner_position(layer, time) == 0:
            return False
    return True
    
def scanner_position(length, time):
    position = time % (2 * length - 2)
    if position > length - 1:
        position = 2 * (length - 1) - position
    return position

find_delay(TEST_INPUT)
find_delay(INPUT)

# [Day 14](http://adventofcode.com/2017/day/14): Disk Defragmentation

In [ ]:
TEST_INPUT = "flqrgnkx"
INPUT = "amgozmfv"
GRID_SIZE = 128

def disk_count(text):
    count = 0
    for i in range(GRID_SIZE):
        row_hash = knot_hash(text + "-" + str(i))
        count += bin(int(row_hash, 16))[2:].count("1")
    return count

assert disk_count(TEST_INPUT) == 8108
disk_count(INPUT)

### Part Two
Now, all the defragmenter needs to know is the number of regions. A region is a group of used squares that are all adjacent, not including diagonals. Every used square is in exactly one region: lone used squares form their own isolated regions, while several adjacent squares all count as a single region.

In the example above, the following nine regions are visible, each marked with a distinct digit:
```
11.2.3..-->
.1.2.3.4   
....5.6.   
7.8.55.9   
.88.5...   
88..5..8   
.8...8..   
88.8.88.-->
|      |   
V      V   
```
Of particular interest is the region marked 8; while it does not appear contiguous in this small view, all of the squares marked 8 are connected when considering the whole 128x128 grid. In total, in this example, 1242 regions are present.

How many regions are present given your key string?

In [ ]:
def disk_graph(text):
    grid = []
    for i in range(GRID_SIZE):
        row_hash = knot_hash(text + "-" + str(i))
        binary = bin(int(row_hash, 16))[2:]
        binary = (GRID_SIZE - len(binary)) * "0" + binary
        grid.append([int(i) for i in list(binary)])
    return grid

def bounded_neighbors(x, y, size):
    neighbors = []
    if x < size - 1:
        neighbors.append((x + 1, y))
    if y < size - 1:
        neighbors.append((x, y + 1))
    if x > 0:
        neighbors.append((x - 1, y))
    if y > 0:
        neighbors.append((x, y - 1))
    return neighbors

def dfs(x, y, grid):
    size = len(grid[0])
    visited = set((x, y))
    to_visit = bounded_neighbors(x, y, size)
    while to_visit:
        x, y = to_visit.pop()
        if grid[x][y] == 1 and (x, y) not in visited:
            visited.add((x, y))
            to_visit.extend(bounded_neighbors(x, y, size))
    
    return visited

def count_regions(grid, size):
    visited = set()
    regions = 0
    for i in range(size):
        for j in range(size):
            if (i, j) not in visited and grid[i][j] == 1:
                seen = dfs(i, j, grid)
                regions += 1
                visited = visited | seen
    return regions
                
TEST_GRID = [[1, 0, 1],
            [0, 0, 1],
            [1, 0, 1]]

assert count_regions(TEST_GRID, 3) == 3
assert count_regions(disk_graph(TEST_INPUT), GRID_SIZE) == 1242

count_regions(disk_graph(INPUT), GRID_SIZE)